# Deploy a Model from a shared Model Package Group

In [ ]:
import json
from sagemaker import ModelPackage
import sagemaker
from sagemaker import get_execution_role
from IPython.core.display import Image, display
from time import gmtime, strftime


In [ ]:
# set required parameters used in cells below
# Hub / Shared Account ID
hub_account_id = 'AWS_ACCOUNT_ID'
# Region used in the Hub account deployments
hub_region = 'AWS_REGION'
# KMS Key id deployed in the Hub account
hub_kms_key_id = 'KMS_KEY_ID'
# S3 bucket name created in the Hub account (The template HubS3BucketName parameter value)
hub_s3_bucket_name = 'HubS3BucketName parameter value used in Cloudformation'

# derive KMS key ARN
hub_kms_key_arn = 'arn:aws:kms:{hub_region}:{hub_account_id}:key/{hub_kms_key_id}'.format(
    hub_account_id=hub_account_id,
    hub_region=hub_region,
    hub_kms_key_id=hub_kms_key_id
)

# set the s3 bucket full name
hub_s3_bucket = 'sagemaker-{hub_region}-{hub_account_id}-{hub_s3_bucket_name}'.format(
    hub_region=hub_region,
    hub_account_id=hub_account_id,
    hub_s3_bucket_name=hub_s3_bucket_name
)

In [ ]:
shared_model_bucket_name = hub_s3_bucket
kms_key_id = hub_kms_key_arn

from botocore.client import ClientError
import boto3
s3_client = boto3.resource('s3')
bucket = s3_client.Bucket(shared_model_bucket_name)
try:
    s3_client.meta.client.head_bucket(Bucket=shared_model_bucket_name)
    print("The bucket "+shared_model_bucket_name+" exists and you have access.")
except ClientError:
    print("The bucket "+shared_model_bucket_name+" does not exist or you have no access.")
    raise

kms_client = boto3.client('kms')
try:
    response = kms_client.describe_key(
        # An identifier for the KMS key. You can use the key ID, key ARN, alias name, alias ARN of the KMS key.
        KeyId=kms_key_id,
    )
    print("The KMS "+kms_key_id+" key exists and you have access.")
except ClientError:
    print("The KMS key "+kms_key_id+" does not exist or you have no access.")
    raise

In [ ]:
role = get_execution_role()
sagemaker_session = sagemaker.Session()
boto3 = sagemaker_session.boto_session
bucket = shared_model_bucket_name
region = sagemaker_session.boto_region_name

s3 = boto3.client("s3")
runtime = boto3.client("runtime.sagemaker")

In [ ]:
sagemaker_client = boto3.client("sagemaker")

In [ ]:
response = sagemaker_client.list_model_package_groups(CrossAccountFilterOption="CrossAccount")
print(response)
model_package_group_arn = response['ModelPackageGroupSummaryList'][0]['ModelPackageGroupArn']
print(model_package_group_arn)
model_package_group_name = response['ModelPackageGroupSummaryList'][0]['ModelPackageGroupName']
print(model_package_group_name)

In [ ]:
real_time_inference_instance_type = "ml.m5.xlarge"

In [ ]:
response = sagemaker_client.list_model_package_groups(CrossAccountFilterOption="CrossAccount")
model_package_group_name = response['ModelPackageGroupSummaryList'][0]['ModelPackageGroupName']
print("Found Model Package Group with name",model_package_group_name)

response = sagemaker_client.list_model_packages(
    # NameContains='string',
    ModelApprovalStatus='Approved',
    ModelPackageGroupName=model_package_group_arn,
    # ModelPackageType='Versioned',
    SortBy='CreationTime',
    SortOrder='Descending'
)
# print(model_package_group_name,response)

if len(response['ModelPackageSummaryList'])>0:
    model_version_arn = response['ModelPackageSummaryList'][0]['ModelPackageArn']
else:
    print("Could not find a model in Approved state")
    raise

print("Selected latest created and approved model, with model_version_arn:",model_version_arn)

In [ ]:
container_list = [{'ModelPackageName': model_version_arn}]
container_list

#### Before to be able to create a model, you need to move the model status to "Approved" in the Shared Registry

In [ ]:
model_name = "CreditRisk"

In [ ]:
container_list = [
    {
        'ModelPackageName': model_version_arn
    }
]

create_model_response = sagemaker_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = container_list,
)
print("Model arn : {}".format(create_model_response["ModelArn"]))

In [ ]:
endpoint_config_name = 'DEMO-CreditRisk-EndpointConfig-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    KmsKeyId=kms_key_id,  # This is the correct place to specify the KMS key  
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

### Deploy the model to an endpoint

In [ ]:
endpoint_name = 'DEMO-CreditRisk-endpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

create_endpoint_response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

## Resources Cleanup

In [ ]:
break # prevent running cells below automatically when running all cells at once

In [ ]:
sagemaker_client.delete_model(
    ModelName = model_name,
)

In [ ]:
sagemaker_client.delete_endpoint_config(
    EndpointConfigName = endpoint_config_name
)

In [ ]:
sagemaker_client.delete_endpoint(
    EndpointName=endpoint_name
)